En esta sección se muestra el proceso de adquisición de los datos (importados desde un archivo excel), la creación de las variables objetivo que se necesitan para generar los modelos y la preparación de los datos para generar el ABT necesario para la construcción de los modelos predictivos.

## Librerías utilizadas

In [1]:
import pandas as pd
import numpy as np

# Para poder visualizar todas las columnas:
pd.set_option('display.max_columns',100)

# Para ver unicamente 3 decimales:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Acceso a los datos

Se importan los datos desde un archivo csv y se visualizan los primeros registros para corroborar que se importaron correctamente:

In [2]:
data_inicial = pd.read_csv('C:/Users/gusta/OneDrive/Documentos/Tesis/Datos/Datos.csv') 
print(data_inicial.head())

   ID_EVENTO SEXO  EDAD  SMet    PAD     PAS  GLU  HDL  TRI     CC   PESO  \
0     623823    F    20     0 80.000 130.000   88   51   64 95.000 73.940   
1     623827    M    19     1 80.000 140.000   89   47  207 94.000 80.050   
2     623848    F    22     0 80.000 110.000   84   68   90 67.000 52.110   
3     623899    F    20     0 70.000 110.000   80   71   87 74.000 54.910   
4     623908    M    19     0 74.000 124.000  113   44  101 95.000 91.990   

    TALLA    IMC  C_TOTAL  HOMA    INS  CC_AGUA  CC_GRAS  CC_HUES  CC_MUSC  \
0 156.000 30.383      144 1.304  6.000   30.600   32.400    2.740    8.200   
1 166.500 28.876      161 3.428 15.600   39.700   26.100    3.550   10.700   
2 158.000 20.874      201 0.830  4.000   28.100   13.600    2.810    7.600   
3 157.000 22.277      183 0.751  3.800   32.000   11.800    2.810    8.300   
4 174.000 30.384      152 3.683 13.200   50.700   23.400    3.890   14.000   

   P_AGUA  P_GRAS  P_HUES  P_MUSC     VFA  
0  41.385  43.819   3.70

## Filtrado de variables 

Se seleccionan unicamente las medidas (variables) que no provienen de la química sanguínea, excepto las variables TRI y HDL, dado que la premisa es construir los modelos predictivos usando los valores de las medidas no invasivas.

In [3]:
data = data_inicial[[x for x in data_inicial.columns if x not in ['C_TOTAL','HOMA','INS']]]

Se calcula la Presión Arterial Media (PAM) la cual ayuda a resumir la información de dos variables (PAD y PAS):

In [4]:
data.insert(6,'PAM', (data.PAS + 2*data.PAD)/3)

## Perfilamiento de los datos

La cantidad de pacientes que presentan síndrome metabólico (SMet=1) vs aquellos que no lo tienen (SMet=0):

In [5]:
conteos = data['SMet'].value_counts()
porcentaje = data['SMet'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'Conteo': conteos, 'Porcentaje': porcentaje})

,Conteo,Porcentaje
0,1550,87.82%
1,215,12.18%


Se calculan los principales estadísticos a cada una de las variables analizadas a través de un método incluido en la librería de Pandas:

In [6]:
data.describe()

,ID_EVENTO,EDAD,SMet,PAD,PAS,PAM,GLU,HDL,TRI,CC,PESO,TALLA,IMC,CC_AGUA,CC_GRAS,CC_HUES,CC_MUSC,P_AGUA,P_GRAS,P_HUES,P_MUSC,VFA
count,1765.000,1765.000,1765.000,1758.000,1758.000,1758.000,1765.000,1765.000,1765.000,1758.000,1765.000,1759.000,1759.000,1765.000,1765.000,1765.000,1765.000,1765.000,1765.000,1765.000,1765.000,1765.000
mean,1902077.852,18.999,0.122,71.776,106.935,83.496,90.569,48.965,106.573,81.359,62.718,162.374,23.706,32.203,18.755,3.048,8.712,51.782,29.311,4.910,13.997,60.007
std,600394.926,1.438,0.327,8.729,11.740,8.853,8.177,10.310,56.874,11.191,13.330,8.261,4.235,7.021,8.684,0.625,1.952,7.220,9.768,0.640,1.991,34.360
min,623823.000,18.000,0.000,40.000,64.000,48.000,60.000,17.000,22.000,42.500,38.370,137.000,14.889,17.300,1.500,1.470,4.800,33.779,2.703,2.603,9.167,5.000
25%,1793376.000,18.000,0.000,66.000,100.000,77.333,86.000,42.000,71.000,73.500,52.930,156.000,20.627,26.900,12.600,2.590,7.200,46.546,22.504,4.488,12.561,38.200
50%,2095692.000,18.000,0.000,70.000,110.000,83.333,90.000,48.000,91.000,80.000,60.370,162.000,22.923,30.300,17.400,2.910,8.200,51.219,30.049,4.936,13.806,56.800
75%,2120448.000,19.000,0.000,80.000,112.000,90.000,95.000,55.000,126.000,87.500,70.040,168.000,26.201,36.900,23.800,3.420,10.000,56.691,36.391,5.358,15.342,78.300
max,5642826.000,24.000,1.000,120.000,160.000,127.333,220.000,113.000,698.000,126.000,127.900,188.000,43.415,67.800,65.300,7.060,19.100,74.403,53.931,9.431,20.650,479.600


Del procedimiento anterior se identifica que hay registros en los que se tienen missing values en alguna de las variables. A continuación, se filtraran esos registros para que sean analizados:

In [7]:
nulos = data.isnull().values.any(axis=1)
data_nulos = data[nulos]
data_nulos

,ID_EVENTO,SEXO,EDAD,SMet,PAD,PAS,PAM,GLU,HDL,TRI,CC,PESO,TALLA,IMC,CC_AGUA,CC_GRAS,CC_HUES,CC_MUSC,P_AGUA,P_GRAS,P_HUES,P_MUSC,VFA
169,911364,F,18,0,nan,nan,nan,87,43,175,nan,80.710,nan,nan,31.300,37.900,2.910,8.600,38.781,46.958,3.606,10.655,71.600
216,1159105,F,19,0,nan,nan,nan,91,47,188,nan,51.950,nan,nan,24.800,18.000,2.350,6.800,47.738,34.649,4.524,13.090,62.300
509,1793514,F,23,0,nan,nan,nan,93,56,82,nan,73.790,nan,nan,29.000,34.100,2.990,7.700,39.301,46.212,4.052,10.435,96.400
631,1794686,M,18,0,nan,nan,nan,99,65,80,81.000,67.130,171.000,22.957,39.800,12.600,3.730,11.000,59.288,18.770,5.556,16.386,36.200
1520,2120717,F,18,0,nan,nan,nan,82,43,58,nan,59.560,nan,nan,29.800,19.000,2.960,7.800,50.034,31.901,4.970,13.096,53.900
1527,2120727,F,18,0,nan,nan,nan,89,54,57,nan,48.170,nan,nan,26.500,12.000,2.670,7.000,55.013,24.912,5.543,14.532,19.800
1529,2120729,F,18,0,nan,nan,nan,88,32,135,nan,60.000,nan,nan,27.300,22.700,2.700,7.300,45.500,37.833,4.500,12.167,70.800
1583,2120804,F,21,0,60.000,110.000,76.667,72,38,130,nan,59.470,158.000,23.822,29.100,19.800,2.770,7.800,48.932,33.294,4.658,13.116,67.700


Se identifica que todos los pacientes en los que se tienen missing values no tienen síndrome metabólico y que este subconjunto de pacientes representa menos del 1% de la población total, por lo que eliminar aquellos registros con missing values no representará una pérdida significativa de información y de esta forma no se provoca un ningún sesgo (por más mínimo que sea) en el análisis al utilizar algún método de imputación de valores:

In [8]:
data = data.dropna()
conteos = data['SMet'].value_counts()
porcentaje = data['SMet'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'Conteo': conteos, 'Porcentaje': porcentaje})

,Conteo,Porcentaje
0,1542,87.76%
1,215,12.24%


Se calculan nuevamente los principales estadísticos a cada una de las variables consideradas:

In [9]:
data.describe()

,ID_EVENTO,EDAD,SMet,PAD,PAS,PAM,GLU,HDL,TRI,CC,PESO,TALLA,IMC,CC_AGUA,CC_GRAS,CC_HUES,CC_MUSC,P_AGUA,P_GRAS,P_HUES,P_MUSC,VFA
count,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000,1757.000
mean,1902689.677,18.999,0.122,71.783,106.933,83.500,90.583,48.973,106.544,81.360,62.719,162.372,23.707,32.214,18.741,3.049,8.715,51.799,29.288,4.911,14.002,60.008
std,600932.976,1.437,0.328,8.727,11.743,8.854,8.178,10.311,56.910,11.194,13.342,8.262,4.237,7.029,8.680,0.626,1.954,7.219,9.766,0.640,1.990,34.406
min,623823.000,18.000,0.000,40.000,64.000,48.000,60.000,17.000,22.000,42.500,38.370,137.000,14.889,17.300,1.500,1.470,4.800,33.779,2.703,2.603,9.167,5.000
25%,1793376.000,18.000,0.000,66.000,100.000,77.333,86.000,42.000,71.000,73.500,52.930,156.000,20.624,26.900,12.600,2.590,7.200,46.562,22.492,4.488,12.564,38.200
50%,2095692.000,18.000,0.000,70.000,110.000,83.333,90.000,48.000,91.000,80.000,60.400,162.000,22.917,30.400,17.400,2.910,8.200,51.236,30.035,4.936,13.809,56.800
75%,2120446.000,19.000,0.000,80.000,112.000,90.000,95.000,55.000,126.000,87.500,70.040,168.000,26.203,36.900,23.800,3.420,10.000,56.708,36.353,5.358,15.345,78.300
max,5642826.000,24.000,1.000,120.000,160.000,127.333,220.000,113.000,698.000,126.000,127.900,188.000,43.415,67.800,65.300,7.060,19.100,74.403,53.931,9.431,20.650,479.600


## Construcción de variables objetivo

Uno de los criterios para diagnosticar síndrome metabólico es que los triglicéridos estén por arriba de 150 mg/dL:

In [10]:
data['TARGET_TRI'] = np.where(data['TRI']>150,1,0)
conteos_tri = data['TARGET_TRI'].value_counts()
porcentaje_tri = data['TARGET_TRI'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'Conteo': conteos_tri, 'Porcentaje': porcentaje_tri})

,Conteo,Porcentaje
0,1479,84.18%
1,278,15.82%


Otro de los criterios para diagnosticar síndrome metabólico es que el colesterol HDL esté por debajo de 40 mg/dL en hombres y de 50 mg/dL en mujeres. Como no se cuenta con suficientes datos para hacer un modelo para cada segmento, se tomará como punto de corte para ambos sexos que el colesterol HDL esté por debajo de 40 mg/dL.

In [11]:
data['TARGET_HDL'] = np.where(data['HDL']<40 ,1,0)
conteos_hdl = data['TARGET_HDL'].value_counts()
porcentaje_hdl = data['TARGET_HDL'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'Conteo': conteos_hdl, 'Porcentaje': porcentaje_hdl})

,Conteo,Porcentaje
0,1449,82.47%
1,308,17.53%


Para finalizar, se guardarán los cambios hechos en esta sección en un archivo .csv para continuar con su análisis en las demás secciones.

In [12]:
data.to_csv('C:/Users/gusta/OneDrive/Documentos/Tesis/Datos/Datos_prep.csv', index=False)